In [1]:
import tensorflow as tf 
import pandas as pd 
import numpy as np 
import json
from tensorflow import feature_column
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
from tensorboard.plugins.hparams import api as hp
import datetime

## Load Data and Create Dataset

In [2]:
df = pd.read_csv("data/stage_data_out/dataset/Merge_Dataset/Merge_Dataset.csv", index_col=0)
print(df.dtypes)
print(df.describe())
print(df.head(5))

ear_l     float64
ear_r     float64
ear       float64
Target      int64
dtype: object
               ear_l          ear_r            ear         Target
count  116069.000000  116069.000000  116069.000000  116069.000000
mean        0.305900       0.304856       0.305378       0.732986
std         0.033780       0.031586       0.031354       0.442401
min         0.112955       0.093567       0.105054       0.000000
25%         0.286097       0.290172       0.289110       0.000000
50%         0.311300       0.310224       0.312204       1.000000
75%         0.329276       0.325553       0.326692       1.000000
max         0.440620       0.500000       0.427168       1.000000
          ear_l     ear_r       ear  Target
frame                                      
0.0    0.282051  0.289474  0.285762       0
1.0    0.294775  0.310811  0.302793       0
2.0    0.294872  0.281959  0.288415       0
3.0    0.269142  0.289374  0.279258       0
4.0    0.277569  0.276817  0.277193       0


In [3]:
target = df.pop('Target')
dataset = tf.data.Dataset.from_tensor_slices((dict(df), target.values))
print(dataset)


<TensorSliceDataset shapes: ({ear_l: (), ear_r: (), ear: ()}, ()), types: ({ear_l: tf.float64, ear_r: tf.float64, ear: tf.float64}, tf.int64)>


In [4]:
for feature_batch, label_batch in dataset.take(1):
  print('Every feature:', list(feature_batch.keys()))
  print('A batch of ear:', feature_batch['ear'])
  print('A batch of targets:', label_batch )

Every feature: ['ear_l', 'ear_r', 'ear']
A batch of ear: tf.Tensor(0.2857624831309042, shape=(), dtype=float64)
A batch of targets: tf.Tensor(0, shape=(), dtype=int64)


## Splitting, Shuffing, Batching  data

### Splitting and Shuffling

In [5]:
dataset_size = dataset.reduce(0, lambda x, _: x + 1).numpy()
dataset = dataset.shuffle(buffer_size = dataset_size)

train_size = int(0.7*dataset_size)
val_size = int(0.15*dataset_size)
test_size = int(0.15*dataset_size)

train = dataset.take(train_size)
val = dataset.skip(train_size)
val = dataset.take(val_size)
test = dataset.skip(train_size + val_size)
test = dataset.take(test_size)

train_size = train.reduce(0, lambda x, _: x + 1).numpy()
val_size = val.reduce(0, lambda x, _: x + 1).numpy()
test_size = test.reduce(0, lambda x, _: x + 1).numpy()

print("Full dataset size:", dataset_size)
print("Train dataset size:", train_size)
print("Val dataset size:", val_size)
print("Test dataset size:", test_size)

Full dataset size: 116069
Train dataset size: 81248
Val dataset size: 17410
Test dataset size: 17410


### Shuffling, Batching

In [6]:
BATCH_SIZE = 32

train = train.shuffle(buffer_size = train_size)
train = train.batch(BATCH_SIZE)

val = val.shuffle(buffer_size = val_size)
val = val.batch(BATCH_SIZE)

test = test.batch(BATCH_SIZE)

## Feature Engineering

In [7]:
example_batch = next(iter(train))[0]


In [8]:
def demo(feature_column):
  feature_layer = layers.DenseFeatures(feature_column)
  print(feature_layer(example_batch).numpy())

# POnly if we have features with different scale
def normalize_numerical_features(df, features):
  def get_mean_std(x):
    return df[x].mean(), df[x].std()
  for column in features: 
    mean, std = get_mean_std(column)
    def z_score(col):
      return (col - mean)/std    
    def _numeric_column_normalized(column_name, normalizer_fn):
      return tf.feature_column.numeric_column(column_name, normalizer_fn=normalizer_fn)
    return _numeric_column_normalized(column,z_score)
  
def get_normalization_layer(name, dataset):
  # Create a Normalization layer for our feature.
  normalizer = preprocessing.Normalization()
  # Prepare a Dataset that only yields our feature.
  feature_ds = dataset.map(lambda x, y: x[name])
  # Learn the statistics of the data.
  normalizer.adapt(feature_ds)
  return normalizer

def make_numerical_feature_col(numerical_column, normalize = False):
    for column_name in numerical_column:
        numeric_col = tf.keras.Input(shape=(1,), name=column_name)
        if normalize : 
            normalization_layer = get_normalization_layer(column_name, train)
            encoded_numeric_col = normalization_layer(numeric_col) 
        else : 
            encoded_numeric_col = feature_column.numeric_column(column_name)
        all_inputs.append(numeric_col)
        encoded_features.append(encoded_numeric_col)
    return all_inputs, encoded_features


In [9]:
all_inputs = []
encoded_features = []
numerical_features = ["ear","ear_l","ear_r"]
all_inputs, encoded_features = make_numerical_feature_col(numerical_features, normalize = True)

In [10]:
all_features = []
all_features = tf.keras.layers.concatenate(encoded_features)

## Model

## Hyper Parameter tuning (Hparams & tensor board)

### Define log dir

In [11]:
logdir = "tensorboard/logs/fit/tunning/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")+"/"

### Define model parameter

In [20]:
HP_NUM_UNITS_1 = hp.HParam('num_units_1', hp.Discrete([32,64,128,256,512]))
HP_NUM_UNITS_2 = hp.HParam('num_units_2', hp.Discrete([32,64,128,256,512]))
HP_DROPOUT = hp.HParam('dropout', hp.RealInterval(0.2, 0.5))
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam's,'adadelta','sgd']))
HP_ACTIVATION = hp.HParam('activation', hp.Discrete(['relu','elu']))
HP_ACTIVATION_OUTPUT = hp.HParam('activation_output', hp.Discrete(['sigmoid']))


METRIC_BINARY_ACCURACY = "binary_accuracy"
METRIC_BINARY_CROSSENTROPY = "binary_crossentropy"
METRIC_MSE = "mean_squared_error"

NUMBER_OF_TARGET = 1
metrics = ["binary_accuracy","binary_crossentropy","mean_squared_error"]


### Initialize hyper parameter for the log

In [13]:
with tf.summary.create_file_writer(logdir).as_default():
  hp.hparams_config(
    hparams=[HP_NUM_UNITS_1, HP_NUM_UNITS_2, HP_DROPOUT, HP_ACTIVATION, HP_ACTIVATION_OUTPUT, HP_OPTIMIZER],
    metrics=[ hp.Metric(METRIC_BINARY_ACCURACY, display_name='Binary Accuracy'),
              hp.Metric(METRIC_BINARY_CROSSENTROPY, display_name='Binary Cross Entropy'),
              hp.Metric(METRIC_MSE, display_name='MSE'),
    ],
  )

### Define the model

In [14]:
def modeling(hparams):
    
    x = tf.keras.layers.BatchNormalization()(all_features)
    x = tf.keras.layers.Dense(hparams[HP_NUM_UNITS_1],activation=hparams[HP_ACTIVATION])(x)
    x = tf.keras.layers.Dropout(hparams[HP_DROPOUT])(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dense(hparams[HP_NUM_UNITS_2],activation=hparams[HP_ACTIVATION])(x)
    x = tf.keras.layers.Dropout(hparams[HP_DROPOUT])(x)
    x = tf.keras.layers.BatchNormalization()(x)

    output = tf.keras.layers.Dense(NUMBER_OF_TARGET, activation=hparams[HP_ACTIVATION_OUTPUT])(x)
    model = tf.keras.Model(all_inputs,output)
    return model



In [18]:
def train_test_model(hparams):
    model = modeling(hparams)
    model.summary()
    model.compile(
        optimizer = hparams[HP_OPTIMIZER],
        loss = tf.keras.losses.BinaryCrossentropy(),
        metrics = ["binary_accuracy","binary_crossentropy","mean_squared_error"],
    )
    model.fit(
        train, 
        validation_data= val,
        epochs=10,
        shuffle=True,
        verbose =1,
        callbacks=[ 
            tf.keras.callbacks.TensorBoard(logdir),  # log metrics
            hp.KerasCallback(logdir, hparams),  # log hparams
            tf.keras.callbacks.EarlyStopping(monitor='val_binary_crossentropy', patience=10),
        ]
    ) 
    _, binary_accuracy, binary_crossentropy, mean_squared_error = model.evaluate(test)
    return binary_accuracy, binary_crossentropy, mean_squared_error

### Define a method to run the the training and testing model function and logs the paramete

In [16]:
def run(run_dir, hparams):
    with tf.summary.create_file_writer(run_dir).as_default():
        hp.hparams(hparams)  # record the values used in this trial
        binary_accuracy, binary_crossentropy, mean_squared_error = train_test_model(hparams)
        tf.summary.scalar(METRIC_BINARY_ACCURACY, binary_accuracy, step=1)
        tf.summary.scalar(METRIC_BINARY_CROSSENTROPY, binary_crossentropy, step=1)
        tf.summary.scalar(METRIC_MSE, mean_squared_error, step=1)

### Tunning the model

In [21]:
session_num = 0
 
for num_units_1 in HP_NUM_UNITS_1.domain.values:
  for num_units_2 in HP_NUM_UNITS_2.domain.values:
      for dropout_rate in (HP_DROPOUT.domain.min_value, HP_DROPOUT.domain.max_value):
        for optimizer in HP_OPTIMIZER.domain.values:
          for activation in HP_ACTIVATION.domain.values:
            for activation_output in HP_ACTIVATION_OUTPUT.domain.values:
              hparams = {
                HP_NUM_UNITS_1: num_units_1,
                HP_NUM_UNITS_2: num_units_2,
                HP_DROPOUT : dropout_rate,
                HP_OPTIMIZER: optimizer,
                HP_ACTIVATION: activation,
                HP_ACTIVATION_OUTPUT: activation_output
              }
              run_name = "run-%d" % session_num
              print('--- Starting trial: %s' % run_name)
              print({h.name: hparams[h] for h in hparams})
              run(logdir + run_name, hparams)
              session_num += 1          

--- Starting trial: run-0
{'num_units_1': 32, 'num_units_2': 32, 'dropout': 0.2, 'optimizer': 'adadelta', 'activation': 'elu', 'activation_output': 'sigmoid'}
Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
ear (InputLayer)                [(None, 1)]          0                                            
__________________________________________________________________________________________________
ear_l (InputLayer)              [(None, 1)]          0                                            
__________________________________________________________________________________________________
ear_r (InputLayer)              [(None, 1)]          0                                            
__________________________________________________________________________________________________
normalization (Normalization)   

KeyboardInterrupt: 

In [3]:
 %reload_ext tensorboard
 %tensorboard --logdir 'tensorboard/logs/fit'

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 69428), started 0:02:10 ago. (Use '!kill 69428' to kill it.)

In [24]:
!jupyter nbconvert --to script ann_fatiuge_test.ipynb

[NbConvertApp] Converting notebook ann_fatiuge_test.ipynb to script
[NbConvertApp] Writing 8502 bytes to ann_fatiuge_test.py
